In [4]:
import cv2
import numpy as np
from math import log10, sqrt

def is_grayscale(image):
    """
    Check if the image is grayscale.
    """
    if len(image.shape) < 3:
        return True
    elif image.shape[2] == 1:
        return True
    else:
        return False

def makePicture(pic, grayscale):
    img1 = []
    img2 = []
    row = pic[-1][0] + 1
    col = pic[-1][1] + 1
    if grayscale:
        for i in range(row):
            for j in range(col):
                index = i * col + j
                pixel_value = np.uint8(pic[index][2])
                img1.append([pixel_value])
            img2.append(img1)
            img1 = []
        return np.array(img2)
    else:
        for i in range(row):
            for j in range(col):
                index = i * col + j
                img1.append([
                    np.uint8(pic[index][4]),
                    np.uint8(pic[index][3]),
                    np.uint8(pic[index][2])
                ])
            img2.append(img1)
            img1 = []
        return np.array(img2)

def getTextFromFile(filename):
    with open(filename, 'r') as file:
        return file.read()

def getPicture(filename):
    img = cv2.imread(filename, cv2.IMREAD_UNCHANGED)
    if img is None:
        raise ValueError("Image not found or unable to read")
    grayscale = is_grayscale(img)
    piclist = []
    row, col = img.shape[:2]
    if grayscale:
        for i in range(row):
            for j in range(col):
                pix = img[i][j]
                piclist.append([i, j, pix, pix, pix])
    else:
        for i in range(row):
            for j in range(col):
                pix_b = img[i][j][0]
                pix_g = img[i][j][1]
                pix_r = img[i][j][2]
                piclist.append([i, j, pix_r, pix_g, pix_b])
    return piclist, grayscale

def putDataInPixel(index, threeBinary, pixels):

    pixels[index][2] &= 254
    pixels[index][3] &= 254
    pixels[index][4] &= 254

    pixels[index][2] |= int(threeBinary[0], 2)
    pixels[index][3] |= int(threeBinary[1], 2)
    pixels[index][4] |= int(threeBinary[2], 2)

def stg(message, pixels):
    messageLength = len(message)
    messageLengthBin = bin(messageLength)[2:].zfill(24)
    pixelIndex = 0


    for i in range(0, 24, 3):
        threeBits = messageLengthBin[i: i + 3]
        putDataInPixel(pixelIndex, threeBits, pixels)
        pixelIndex += 1


    messageInBin = ''.join([bin(ord(char))[2:].zfill(8) for char in message])


    padding_length = (3 - (len(messageInBin) % 3)) % 3
    messageInBin += '0' * padding_length


    for i in range(0, len(messageInBin), 3):
        threeBits = messageInBin[i:i+3]
        putDataInPixel(pixelIndex, threeBits, pixels)
        pixelIndex += 1

def exportDataFromPixel(index, pixels):

    ch1_bit = bin(pixels[index][2] & 1)[2:]
    ch2_bit = bin(pixels[index][3] & 1)[2:]
    ch3_bit = bin(pixels[index][4] & 1)[2:]
    return ch1_bit + ch2_bit + ch3_bit

def decrypt(pixels):

    msgLenInBinary = ""

    for ind in range(8):
        msgLenInBinary += exportDataFromPixel(ind, pixels)
    msgLenInBinary = msgLenInBinary[:24]
    msgLen = int(msgLenInBinary, 2)

    secretMsgInBinary = ""
    numBits = msgLen * 8
    numPixels = (numBits + 2) // 3


    for i in range(8, 8 + numPixels):
        secretMsgInBinary += exportDataFromPixel(i, pixels)
    secretMsgInBinary = secretMsgInBinary[:numBits]


    secretMsg = ""
    for j in range(0, len(secretMsgInBinary), 8):
        byte = secretMsgInBinary[j:j+8]
        secretMsg += chr(int(byte, 2))
    return secretMsg

def PSNR(original, stego):
    mse = np.mean((original - stego) ** 2)
    if mse == 0:
        return 100
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr

def MSE(original, stego):
    """Calculate the Mean Squared Error (MSE) between two images."""
    mse = np.mean((original - stego) ** 2)
    return mse

def main():
    print("Welcome!")
    print("1. Encrypt a message in an image.")
    print("2. Decrypt a message from an image.")
    print("3. Quit")

    while True:
        try:
            task_number = int(input("Enter number of your wanted task: "))

            if task_number == 1:
                pictureName = input("Enter the image filename to use for encryption (use PNG format): ")
                textFilename = input("Enter the text filename containing the message: ")
                outputImageName = input("Enter the output image filename (use PNG format): ")


                if not pictureName.lower().endswith('.png'):
                    print("Please use a PNG image for encryption to avoid data corruption.")
                    continue
                if not outputImageName.lower().endswith('.png'):
                    print("Please save the encrypted image with a .png extension.")
                    continue


                try:
                    messageText = getTextFromFile(textFilename)
                except FileNotFoundError:
                    print("Text file not found.")
                    continue


                try:
                    pixels, grayscale = getPicture(pictureName)
                    original_image = cv2.imread(pictureName)
                except ValueError as e:
                    print(e)
                    continue


                numAvailablePixels = len(pixels) - 8
                numRequiredPixels = ((len(messageText) * 8) + 2) // 3
                if numRequiredPixels > numAvailablePixels:
                    print("Message is too large to fit in the provided image.")
                    continue


                stg(messageText, pixels)
                encryptedImage = makePicture(pixels, grayscale)
                cv2.imwrite(outputImageName, encryptedImage)
                print(f"Message encrypted and saved in {outputImageName}.")


                stego_image = cv2.imread(outputImageName)
                psnr_value = PSNR(original_image, stego_image)
                print(f"PSNR value: {psnr_value} dB")
                mse_value = MSE(original_image, stego_image)
                print(f"MSE value: {mse_value}")


                difference = cv2.absdiff(original_image, stego_image)
                print("Difference matrix:")
                print(difference)

            elif task_number == 2:
                pictureName = input("Enter the encrypted image filename (use PNG format): ")

                #
                if not pictureName.lower().endswith('.png'):
                    print("Please use a PNG image for decryption to avoid data corruption.")
                    continue


                try:
                    pixels, grayscale = getPicture(pictureName)
                except ValueError as e:
                    print(e)
                    continue


                secretMsg = decrypt(pixels)
                print("Decrypted message:")
                print(secretMsg)

            elif task_number == 3:
                print("Goodbye!")
                break
            else:
                print("Invalid option!")

        except ValueError:
            print("Please enter a valid number.")

if __name__ == "__main__":
    main()
_

Welcome!
1. Encrypt a message in an image.
2. Decrypt a message from an image.
3. Quit
Enter number of your wanted task: 1
Enter the image filename to use for encryption (use PNG format): ct_scan_kidney.png
Enter the text filename containing the message: output_text.txt
Enter the output image filename (use PNG format): Stego_image2.png
Message encrypted and saved in Stego_image2.png.
PSNR value: 82.51318774044562 dB
MSE value: 0.00036455376413371463
Difference matrix:
[[[1 1 1]
  [1 1 1]
  [1 1 1]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
Enter number of your wanted task: 2
Enter the encrypted image filename (use PNG format): Stego_image2.png
Decrypt

''